Plots results of fretting fatigue case from [this paper](https://onlinelibrary.wiley.com/doi/full/10.1002/nme.6067).

Institute Jozef Stefan 2022

# Import libs

In [30]:
import h5py as h5
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable


In [31]:
# To run interactive plot.
from matplotlib import interactive
interactive(True)
%matplotlib qt


# Plot results

In [32]:
def get_vms(sxx, syy, sxy, nu):
    szz = nu * (sxx+syy)
    return np.sqrt(np.power(sxx, 2)+3*np.power(sxy, 2)+np.power(syy, 2)+(-1)*syy*szz+np.power(szz, 2)+(-1)*sxx*(syy+szz))


In [34]:
# Import data.
data = h5.File('results_fatigue.h5', 'r')

pos = np.array(data['domain/pos'][:]) * 1000
disp = data['displacements'][:] * 1000
disp_magnitude = [np.sqrt(x*x + y*y) for x, y in zip(disp[0], disp[1])]
stress = data['stress'][:]
sxx = stress[0]
syy = stress[1]
sxy = stress[2]
vms = get_vms(sxx, syy, sxy, 0.33) * 1e-6

# Create figure.
fig, axs = plt.subplots(2, 1, figsize=(10, 5))

# Displacement field.
ax = axs[0]
coeff = 100
p = ax.scatter(pos[0] + coeff * disp[0], pos[1] +
               coeff * disp[1], c=disp_magnitude, s=10)
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(p, cax=cax, orientation='vertical',
             ticks=np.linspace(min(disp_magnitude), max(disp_magnitude), 5))
ax.grid('both')
ax.set_aspect('equal')
ax.set_title("displacement field")
ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$y$')

# Stress field.
ax = axs[1]
p = ax.scatter(pos[0], pos[1], c=vms, s=10)
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(p, cax=cax, orientation='vertical',
             ticks=np.linspace(min(vms), max(vms), 5), label="MPa")
ax.grid('both')
ax.set_aspect('equal')
ax.set_title('von Mises stress')
ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$y$')

fig.tight_layout()

data.close()
